# Linear Equations and Computer Basics

This is the first in a set of Notebooks that follow along with the book [*Applied Computational Economics and Finance*](https://mitpress.mit.edu/books/applied-computational-economics-and-finance) (Miranda & Fackler 2004).  The goal is to actually perform some of the exercises and examples that appear througout the book to help solidify my comprehension.  It obviously starts simple, but let's see where we end up.

In [1]:
using Distributions

## Matrix Inversion

It only makes sense to start with matrix inversion, which comes up so often.   In order to appreciate that 1) better methods exist and 2) we have stock methods in Julia, let's see what a brute force method looks like.  Though there are multiple ways to skin a cat, note that the following property of an inverted $n \times n$ matrix is true by construction:

$$AB=1$$

where $B=A^{-1}$.  If $n$=3, then the following example holds:

$$\begin{bmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33} \\
\end{bmatrix}
\begin{bmatrix}
b_{11} & b_{12} & b_{13} \\
b_{21} & b_{22} & b_{23} \\
b_{31} & b_{32} & b_{33} \\
\end{bmatrix}=
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 \\
\end{bmatrix}$$

If we expand this into a system of equations, we get the following:

$$a_{11}b_{11} + a_{12}b_{21} + a_{13}b_{31}=1$$
$$a_{21}b_{11} + a_{22}b_{21} + a_{23}b_{31}=0$$
$$a_{31}b_{11} + a_{32}b_{21} + a_{33}b_{31}=0$$

$$a_{11}b_{12} + a_{12}b_{22} + a_{13}b_{32}=0$$
$$a_{21}b_{12} + a_{22}b_{22} + a_{23}b_{32}=1$$
$$a_{31}b_{12} + a_{32}b_{22} + a_{33}b_{32}=0$$

$$a_{11}b_{13} + a_{12}b_{23} + a_{13}b_{33}=0$$
$$a_{21}b_{13} + a_{22}b_{23} + a_{23}b_{33}=0$$
$$a_{31}b_{13} + a_{32}b_{23} + a_{33}b_{33}=1$$

This generalizes to the following conditional expression...

$$\sum_j^n \sum_k^n a_{ij}b_{kl} = \begin{cases} 1,& \text{if } i = l \text{ & } j=k\\
                                                 0,& \text{otherwise}
                                   \end{cases}\\
                                   \forall \quad i,l \in [1,n]$$
                                   
...which is suggestive of an algorithm.  We, of course, do not know the value of any elements in $B$.  Consequently, the approach has been to solve the system by using the constituent equations to isolate elements of $B$.  Let's focus on the first block in which we have a fully identified system.  We can reduce the instances of $b_{kl}$ in the second and third equations.

$$[a_{21}b_{11} + a_{22}b_{21} + a_{23}b_{31}=0] - (\frac{a_{21}}{a_{11}})[a_{11}b_{11} + a_{12}b_{21} + a_{13}b_{31}=1]$$

$$\&$$

$$[a_{31}b_{11} + a_{32}b_{21} + a_{33}b_{31}=0] - (\frac{a_{31}}{a_{11}})[a_{11}b_{11} + a_{12}b_{21} + a_{13}b_{31}=1]$$

These adjustments lead to the modified system...

$$a_{11}b_{11} + a_{12}b_{21} + a_{13}b_{31}=1$$

$$0 + (a_{22} - \frac{a_{21}a_{12}}{a_{11}})b_{21} + (a_{23} - \frac{a_{21}a_{13}}{a_{11}})b_{31} = -\frac{a_{21}}{a_{11}}$$

$$0 + (a_{32} - \frac{a_{31}a_{12}}{a_{11}})b_{21} + (a_{33} - \frac{a_{31}a_{13}}{a_{11}})b_{31} = -\frac{a_{31}}{a_{11}}$$

Another round of folding, this time finding the difference between the second and third equations allows us to reduce the number of unknowns in the third equation down to 1 ($b_{31}$).  This time the ratio we want is the following:

$$\frac{(a_{32} - \frac{a_{31}a_{12}}{a_{11}})}{(a_{22} - \frac{a_{21}a_{12}}{a_{11}})}$$

Which means we are resolving the following expression:

$$[0 + \frac{a_{31}a_{12}}{a_{11}}b_{21} + \frac{a_{31}a_{13}}{a_{11}}b_{31} = -\frac{a_{31}}{a_{11}}] - \frac{(a_{32} - \frac{a_{31}a_{12}}{a_{11}})}{(a_{22} - \frac{a_{21}a_{12}}{a_{11}})}[0 + \frac{a_{21}a_{12}}{a_{11}}b_{21} + \frac{a_{21}a_{13}}{a_{11}}b_{31} = -\frac{a_{21}}{a_{11}}]$$

$$0 + (a_{22} - \frac{a_{21}a_{12}}{a_{11}})b_{21} + (a_{23} - \frac{a_{21}a_{13}}{a_{11}})b_{31} = -\frac{a_{21}}{a_{11}}$$

$$0 + (a_{32} - \frac{a_{31}a_{12}}{a_{11}})b_{21} + (a_{33} - \frac{a_{31}a_{13}}{a_{11}})b_{31} = -\frac{a_{31}}{a_{11}}$$

## LU Factorization (2.1)

The first example is actually not LU, but forward substitution.  The idea being that it is a completely reasonable method (from a computational standpoint) when we are dealing with a triangular matrix.  Suppose we have a linear model:

$$Ax=b$$

where the following dimensions apply:

+ $A (n \times p)$
+ $x (p \times 1)$
+ $b (n \times 1)$

$A$ and $b$ are given, and $A$ is a lower triangular matrix. 

$$A=
\begin{bmatrix}
a_{11} &         &        &           & 0  \\
a_{21} & a_{22} &        &           &    \\
a_{31} & a_{32} & \ddots &           &    \\
\vdots  & \vdots  & \ddots & \ddots    &    \\
a_{n1} & a_{n2} & \ldots & a_{nn-1} & a_{nn}
\end{bmatrix}$$

How does one determine the solution vector $x$?  Let's try this forward substitution business out.  The algorithm solves for $x$ one position at a time.  It can do so because the equation in the first row only has one unknown.  

$$a_{11}x_1=b_1$$

The second row equation has two unknowns, but one of them was calculated in the first row.  

$$a_{21}x_1 + a_{22}x_2 = b_2$$

Each new row introduces only one new unknown.

$$x_1 = b_1/a_{11}$$

$$x_2=(b_2-a_{21}x_1)/a_{22}$$

$$x_3=(b_3-a_{31}x_1-a_{32}x_2)/a_{33}$$

$$\vdots$$

$$x_n=(b_n - a_{n1}x_1 - a_{n2}x_2 - \cdots - a_{n,n-1}x_{n-1})/a_{nn}$$

This progression generalizes to the following expression

$$x_i = (b_i - \sum_{j=1}^{i-1} a_{ij} x_j)/a_{ii}$$

In [2]:
#Define extent of solution vector
n=5

#Generate a matrix of random positive integers
A=float(rand(1:10,n,n))

#Coerce it into a lower triangular by zero sub
for i=1:n
    for j=1:n
        if j>i
            A[i,j]=0
        end
    end
end

#Generate a response vector of random positive ints
b=float(rand(1:10,n))

# Generate array to hold solution
# x=[0. for i=1:n]

#Define function to solve this system by forward substitution
function fsub(A,b)
    #Generate array to hold solution
    x=[0. for i=1:n]
    #Capture the first value
    x[1]=b[1]/A[1,1]
    #For each equation...
    for i=2:length(b)
        #...solve for x (returns a 1x1 array)...
        tmp=(b[i] - A[i,1:i-1]*x[1:i-1])/A[i,i]
        #...and insert into x
        x[i]=tmp[1]
    end
    return x
end

#Calculate x
x=fsub(A,b)

#Define function to check the solution vector
function csvec(vec1,vec2)
    return all([isapprox(vec1[i],vec2[i]) for i in 1:length(b)])
end

csvec(A*x,b)

true

That was pretty straightforward (once the syntax issues were ironed out).  The algebraic simplicity of this method makes it attractive from a computational standpoint.  The thing is, most matrices in practice do not share this triangular format.  One way to get them there is to use *LU Factorization* to decompose a matrix into upper and lower triangulars.  Then we can use both back and forward substitution, respectively, to solve for the solution vector $x$.  This approach can be used for all square, nonsingular matrices.

There are two phases.  *Factorization* involves the use of Gaussian elimination to factor $A$ into $L$ and $U$.

$$A=LU$$

Note that $L$ will be row-permuted.  Once factored, we are solving the following system.

$$Ax=LUx=L(Ux)=b$$

This arrangement permits a straightforward two-step protocol:

1. Solve for $y$ in $Ly=b$; and,
2. Solve for $x$ in $Ux=y$.

First,let's design a function that performs LU factorization.

In [3]:
#Generate a matrix of random positive integers
A=float(rand(1:10,n,n))

function LU_fact(A)
    #Generate containers for LU factors
    L=eye(n)
    U=copy(A)

    #For each column...
    for j=1:n
        #...and each row below the diagonal...
        for i=j+1:n
            #...capture the ratio of the diagonal value to lower row value in that column...
            tmp_ratio=U[i,j]/U[j,j]
            #...store the ratio in L at the lower row value position...
            L[i,j]=tmp_ratio
            #...and then use the ratio as a multiplier when subtracting the diagonal row from the lower row
            #and assign it to the corresponding row in U
            U[i,1:n]=U[i,1:n]-tmp_ratio*U[j,1:n]
        end
    end
    return (L,U)
end

L,U=LU_fact(A)

csvec(L*U,A)

true

Now we solve for $y=Ux$, and ultimately for $x$.  Since $U$ is an upper triangular matrix, we need to use backward substitution in the second stage equation.  Backward substition is just the opposite of forward substitution.  This time, we are operating on an upper triangular matrix.

$$A =
\begin{bmatrix}
a_{11} & a_{12} & a_{13} & \ldots & a_{1n}  \\
        & a_{22} & a_{23} & \ldots & a_{2n}  \\
        &         & \ddots  & \ddots & \vdots   \\
        &         &         & \ddots & a_{n-1,n}\\
  0     &         &         &        & a_{nn}
\end{bmatrix}$$


The first equation starts with the last row.

$$a_{nn}x_n=b_n$$

The second row equation has two unknowns, but one of them was calculated in the first row.  

$$a_{n-1,n}x_n + a_{n-1,n-1}x_{n-1} = b_{n-1}$$

Each new row introduces only one new unknown.

$$x_n = b_n/a_{nn}$$

$$x_{n-1}=(b_{n-1}-a_{n-1,n}x_n)/a_{n-1,n-1}$$

$$x_{n-2}=(b_{n-2}-a_{n-2,n}x_n-a_{n-2,n-1}x_{n-1})/a_{n-2,n-2}$$

$$\vdots$$

$$x_1=(b_1 - a_{1n}x_n - a_{1,n-1}x_{n-1} - \cdots - a_{12}x_2)/a_{11}$$

This progression generalizes to the following expression

$$x_i = (b_i - \sum_{j=i+1}^{n} a_{ij} x_j)/a_{ii}$$

In [4]:
#Define function to solve this system by backward substitution
function bsub(A,b)
    #Capture length of solution vector
    nv=size(A)[1]
    #Generate array to hold solution
    x=[0. for i=1:nv]
    #Capture the last value value
    x[nv]=b[nv]/A[nv,nv]
    #For each equation...
    for i=nv-1:-1:1
        #...solve for x (returns a 1x1 array)...
        tmp=(b[i] - A[i,i+1:nv]*x[i+1:nv])/A[i,i]
        #...and insert into x
        x[i]=tmp[1]
    end
    return x
end

#Generate response vector
b=float(rand(1:10,n))

#Solve for y
y=fsub(L,b)

#Solve for x
x=bsub(U,y)

println("Ly=b:",csvec(L*y,b))
println("Ux=y:",csvec(U*x,y))
println("Ax=b:",csvec(A*x,b))

Ly=b:true
Ux=y:true
Ax=b:true


Now we are well placed to capture an LU solution function all at once.

In [5]:
function LU_solve(A,b)
    #Factor A
    L,U=LU_fact(A)
    #Solve for y
    y=fsub(L,b)
    #Solve for x
    x=bsub(U,y)
    return x
end

#Generate a matrix of random positive integers
A=float(rand(1:10,n,n))

#Generate response vector
b=float(rand(1:10,n))

#Solve for x
x=LU_solve(A,b)

println("Ax=b:",csvec(A*x,b))

Ax=b:true


## Cholesky Factorization

The LU decomposition is a handy method to be sure (and general), but it can be quite expensive computationally.  When the properties of a matrix permit, we can use more efficient methods.  Specifically, when the square matrix $A$ is [Hermitian](https://en.wikipedia.org/wiki/Hermitian_matrix) [positive definite](https://en.wikipedia.org/wiki/Positive-definite_matrix), we can use [Cholesky factorization](https://en.wikipedia.org/wiki/Cholesky_decomposition).

Let's unpack this a bit:

+ A Hermitian matrix is one in which symmetric entries are complex conjugates of each other (that is $a_{ij} = \overline{a_{ji}}$).  Complex conjugates are just numbers that have equivalent real parts and equal but opposite imaginary parts (e.g. 4 & 4, or 4+2$i$ & 4-2$i$).  Symmetric real matrices, therefore, are a subset.
+ A positive, definite matrix is any Hermitian $n \times n$ matrix $M$ for which $z^TMz$ is a positive scalar ($z$ being any real or complex non-zero column vector of length $n$).  [Intuitively](http://math.stackexchange.com/questions/9758/intuitive-explanation-of-a-positive-semidefinite-matrix), any vector multiplied by a positive definite matrix cannot have it's angle changed by more than 90 degrees.  It is analogous to multiplying a real variable by a positive number.  It can stretch or contract the number, but it cannot reflect it about the origin.

The actual Cholesky decomposition is as follows:

$$A=U^TU$$

$$
\begin{bmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33} \\
\end{bmatrix} = 
\begin{bmatrix}
u_{11} & 0 & 0 \\
u_{12} & u_{22} & 0 \\
u_{13} & u_{23} & u_{33} \\
\end{bmatrix}
\begin{bmatrix}
u_{11} & u_{12} & u_{13} \\
0 & u_{22} & u_{23} \\
0 & 0 & u_{33} \\
\end{bmatrix}$$

where $U$ is upper triangular.  The matrix view makes it easier to see what's happening.  The value of $a_{11}$ is straightforward since there is only single value in the column of $U$ and a single value in the row of its transpose:

$$a_{11}=u_{11}u_{11}+0+0$$
$$\rightarrow u_{11} = +\sqrt{a_{11}}$$

In fact, once we have $u_{11}$ the entire first row can be calculated in a similar manner.

$$a_{1j}=u_{11}u_{1j}+0+0$$
$$\rightarrow u_{1j} = \frac{a_{1j}}{u_{11}}$$

Once the first row of $U$ is known, we can compute $u_{22}$ and use those first row values in to the second row equations:

$$a_{22}=u_{12}^2+u_{22}^2$$
$$\rightarrow u_{22}=+\sqrt{a_{22} - u_{12}^2}$$
$$\rightarrow u_{2j} = \frac{a_{2j} - u_{12}u_{1j}}{u_{22}}$$

The same can be done with the third row.

$$a_{33}=u_{13}^2+u_{23}^2+u_{33}^2$$
$$\rightarrow u_{33}=+\sqrt{a_{33} - u_{13}^2 - u_{23}^2}$$
$$\rightarrow u_{3j} = \frac{a_{3j} - u_{13}u_{1j} - u_{23}u_{2j}}{u_{33}}$$

The pattern emerging here is a two-step operation for each row.

1. $$u_{nn} = +\sqrt{a_{nn} - \sum_{i=1}^{n-1} u_{in}^2}$$
2. $$u_{nj} = \frac{a_{nj} - \sum_{i=1}^{n-1} u_{in}u_{ij}}{u_{nn}}$$


These two formulae allow us to completely construct the Cholesky factor $U$.  Once this decomposition has been performed, we proceed in much the same way as we did with the LU factorization.

$$Ax = U^TUx = U^T(Ux) = U^Ty = b$$

$$Ux = y$$

Now that we have seen the Cholesky algorithm, take note of its vulnerability.  Any time the value under the square root in the calculation of $u_{nn}$ is negative or zero, the algorithm fails.  If it is negative, we end up with a complex number.  If it is zero, unique solutions are no longer ensured.  This is why we know the starting matrix $A$ must be positive definite, and conversely, why Cholesky decomposition is a valid test for positive definiteness.  

Let's go ahead and write a function that performs Cholesky decomposition.

In [6]:
function chol_factor(A)
    #Capture extent of A row dimension
    row_n=size(A)[1]
    #Generate repository for Cholesky factor    
    U=zeros(row_n,row_n)
    #For each row...
    for n=1:row_n
        #...calculate square of the diagonal element of U...
        diag2=A[n,n]-dot(U[1:n-1,n],U[1:n-1,n])
        #...assign the diagonal element of U...
        U[n,n]=sqrt(diag2)
        #...and for each column...
        for j=1:row_n
            #...if we are above the diagonal...
            if n<j
                #...calculate and assign the upper triangular element of U....
                U[n,j]=(A[n,j] - dot(U[1:n-1,n],U[1:n-1,j]))/U[n,n]
            else
                #...otherwise leave it as zero...
                skip
            end
        end
    end
    return U
end

chol_factor (generic function with 1 method)

Again, Cholesky decomposition only works on hermitian positive definite matrices.  This [SE answer](http://math.stackexchange.com/questions/357980/matlab-code-for-generating-random-symmetric-positive-definite-matrix) provides an efficient method of generating such matrices.  First, generate a symmetric matrix by adding a random matrix $A$ to its transpose $A^T$.  Then, impose diagonal dominance by adding a diagonal matrix whose elements equal the extent of the either matrix dimension.  Diagonal dominance is given when the diagonal elements exceed the sum of all off diagonal elements (in both the row and column dimensions).

$$|A_ii| > \sum_{i=1}^n |A_ij| \quad\forall i \neq j$$

In [7]:
function hpd_mat(n)
    #Generate random matrix
    A=float(rand(n,n))
    #Use it to construct a symmetric matrix
    A+=transpose(A)
    #Impose diagonal dominance
    A+=n*eye(n)
    return A
end

test_hpd=hpd_mat(5)

5x5 Array{Float64,2}:
 6.86924   0.770718  0.335413  0.979306  0.628323
 0.770718  6.15186   1.25822   0.259474  0.781309
 0.335413  1.25822   5.77035   1.00633   1.30126 
 0.979306  0.259474  1.00633   5.27447   0.419696
 0.628323  0.781309  1.30126   0.419696  6.34056 

As it turns out, this doesn't ensure a positive definiteness with probability 1, but it's close enough to test our function.  If we wanted to ensure an HPD matrix, we could use the fact that such matrices must have all positive eigenvalues.  Meh...

In [8]:
chol_factor(test_hpd)

5x5 Array{Float64,2}:
 2.62092  0.294063  0.127975  0.373649  0.239733 
 0.0      2.4628    0.49561   0.060743  0.288619 
 0.0      0.0       2.34699   0.395576  0.480421 
 0.0      0.0       0.0       2.2304    0.0549432
 0.0      0.0       0.0       0.0       2.44253  

Ok, does our function work?

In [9]:
#Generate HPD matrix
A=hpd_mat(n)

#Generate Cholesky factor
U=chol_factor(A)

println("U'U=A:",csvec(transpose(U)*U,A))

U'U=A:true


## Iterative Methods

Another situation which can arise is the need to find solutions when $A$ is both large and sparse.  Using Gaussian elimination in this instance leads to the evaluation of a lot of calculations full of zeros, which is not all that useful since those components drop out anyway.  In this situation, a numerical solution can be used to get a solution vector of arbitrary precision.  Consider the following system:

$$Ax=b$$

We can expand this equation via the inclusion of an additional term $Qx$.

$$Ax + Qx = b + Qx$$
$$Qx = b + Qx - Ax$$
$$Qx = b + (Q-A)x$$
$$x = Q^{-1}b + (I - Q^{-1}A)x$$

This expression is equivalent to the original version, but now $x$ is on both sides.  Moreover, it looks eerily familiar.  Behold!

$$x_{n+1} = x_0 + \frac{f(x_0)}{f(x_0)'}$$

That's right, its the old reliable [Newton-Rhapson](https://en.wikipedia.org/wiki/Newton%27s_method) opti algorithm.  One could also go with the classic equation of a line.

$$y = b + mx$$
$$\rightarrow x_1 = x_0 + \frac{dy}{dx}x$$

Anyway, you get the gist.  We are starting at one location, and using the product of $\delta x$ and the gradient to calculate the next number.  In the Newton context, once that gradient flattens out, we are hitting some inflection point and the search slows to a crawl (or stops entirely).  Within that context, we can add some notation to our iterative expression to make it clearer.

$$x^{(k+1)} \leftarrow Q^{-1}b + (I - Q^{-1}A)x^{(k)}$$

The *splitting matrix* $Q$ must satisfy two conditions:

1. $Q^{-1}b$ and $Q^{-1}A$ must be easy to compute, which is ensured if $Q$ is either diagonal or triangular.
2. We want to converge quickly, which is guaranteed from any value of $|| I-Q^{-1}A|| < 1$.  Indeed, the smaller $|| I-Q^{-1}A||$ gets, the faster we converge.

The two most popular methods for this approach are the [Gauss-Jacobi](https://en.wikipedia.org/wiki/Jacobi_method) and [Gauss-Seidel](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method) methods.  The former uses a diagonal $Q$ that consists of the diagonal elements of $A$.  The latter uses an upper triangular $Q$ formed from the upper triangular elements of $A$.  *Both are guaranteed to converge from any starting value of A is diagonally dominant.*

Let's design a function that utilizes the Gauss-Jacobi method.

In [10]:
function diag_mat(A)
    n=size(A)[1]
    dA=eye(n)
    for i=1:n
        dA[i,i]=diag(A)[i]
    end
    return dA
end

function triang_mat(A)
    n=size(A)[1]
    tA=copy(A)
    for i=1:n
        for j=1:n
            if j<i
                tA[i,j]=0
            end
        end
    end
    return tA
end

function iter_jacobi(A,b,x0)
    #Generate Q
    Q=diag_mat(A)

LoadError: syntax: incomplete: "function" at In[10]:23 requires end
while loading In[10], in expression starting on line 23

In [11]:
A

5x5 Array{Float64,2}:
 5.95792   0.918513  0.644198  0.355303  1.06498 
 0.918513  6.54917   1.43549   1.27905   0.526112
 0.644198  1.43549   5.24382   1.06575   0.947997
 0.355303  1.27905   1.06575   5.30211   1.39911 
 1.06498   0.526112  0.947997  1.39911   5.93972 

In [12]:

    

triang_mat(A)

5x5 Array{Float64,2}:
 5.95792  0.918513  0.644198  0.355303  1.06498 
 0.0      6.54917   1.43549   1.27905   0.526112
 0.0      0.0       5.24382   1.06575   0.947997
 0.0      0.0       0.0       5.30211   1.39911 
 0.0      0.0       0.0       0.0       5.93972 

In [13]:
diag(A)*eye(size(A)[1])

LoadError: DimensionMismatch("*")
while loading In[13], in expression starting on line 1